In [ ]:
# default_exp autoencoder.model

# autoencoder.model

> API details.

In [ ]:
# export
import numpy as np
import torch
from torch import nn
from fastrenewables.tabular.model import *
from fastrenewables.timeseries.model import *
from fastai.tabular.all import *
from torch.autograd import Variable


In [ ]:
ann_structure = [10,2]

In [ ]:
# export
class Autoencoder(nn.Module):
    def __init__(self,encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def encode(self, categorical_data, continuous_data, as_np=False):
        z = self.encoder(categorical_data, continuous_data)
        
        if as_np: return to_np(z)
        else: return z
        
    
    def decode(self, categorical_data, continuous_data, as_np=False):
        x = self.decoder(categorical_data, continuous_data)
        
        if as_np: return to_np(x)
        else: return x
        
    def forward(self, categorical_data, continuous_data):
        x = self.encode(categorical_data, continuous_data)
        x = self.decode(categorical_data, x)
        
        return continuous_data

In [ ]:
ae = Autoencoder(MultiLayerPerceptron(ann_structure), MultiLayerPerceptron(ann_structure[::-1]))

In [ ]:
ae.encoder

MultiLayerPerceptron(
  (final_activation): Identity()
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [ ]:
x = torch.randn((3,10), requires_grad=True)
yhat = ae(None, x)
yhat.requires_grad

True

In [ ]:
yhat

tensor([[ 1.3278,  1.4655, -0.5048, -1.6327,  0.3967,  0.1263, -0.7591, -0.2791,
         -2.2149,  0.4619],
        [ 0.5561, -0.3939, -1.8063, -0.8131,  0.4049,  0.0339, -1.3636,  0.3701,
          1.2007, -0.1760],
        [-0.6178,  2.6330,  1.0283, -0.7023,  0.1291,  0.1873,  1.1784,  1.0725,
          1.4288,  2.4448]], requires_grad=True)

In [ ]:
ae_tcn = Autoencoder(TemporalCNN(ann_structure), TemporalCNN(ann_structure[::-1]))
ae_tcn

Autoencoder(
  (encoder): TemporalCNN(
    (bn_cont): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layers): TemporalConvNet(
      (temporal_blocks): Sequential(
        (0): ResidualBlock(
          (conv1): Conv1d(10, 2, kernel_size=(3,), stride=(1,), padding=(2,))
          (chomp1): Chomp1d()
          (act_func1): Identity()
          (dropout1): Dropout2d(p=0.0, inplace=False)
          (conv2): Conv1d(2, 2, kernel_size=(3,), stride=(1,), padding=(2,))
          (chomp2): Chomp1d()
          (act_func2): Identity()
          (dropout2): Dropout2d(p=0.0, inplace=False)
          (net): Sequential(
            (0): Conv1d(10, 2, kernel_size=(3,), stride=(1,), padding=(2,))
            (1): Chomp1d()
            (2): Identity()
            (3): Dropout2d(p=0.0, inplace=False)
            (4): Conv1d(2, 2, kernel_size=(3,), stride=(1,), padding=(2,))
            (5): Chomp1d()
            (6): Identity()
            (7): Dropout2d(p=0.0, inplac

In [ ]:
x = torch.randn((3,10,2), requires_grad=True)
yhat = ae_tcn(None, x)
yhat.requires_grad, yhat.shape

(True, torch.Size([3, 10, 2]))

In [ ]:
yhat[0]

tensor([[-0.2362,  2.1993],
        [ 0.9431, -0.9357],
        [-0.3133, -1.1119],
        [ 0.0310,  0.6986],
        [ 0.1010, -0.1660],
        [-0.3526, -2.0143],
        [-0.1462, -0.6417],
        [ 0.0596,  0.4791],
        [-1.0819,  1.0265],
        [ 1.7406, -0.9789]], grad_fn=<SelectBackward>)

In [ ]:
class UnFlatten(nn.Module):
#     def __init__(self, size):
#         self.size = size
        
    def forward(self, input, dims):
        return input.view(*dims)

In [ ]:
class VariationalAutoencoder(Autoencoder):
    def __init__(self, encoder, decoder, h_dim, z_dim):
        super().__init__(encoder, decoder)
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.flatten = Flatten()
        self.unflatten = UnFlatten()
        
        self.hidden2mu = nn.Linear(h_dim, z_dim)
        self.hidden2logvar = nn.Linear(h_dim, z_dim)
        
    def encode(self, categorical_data, continuous_data, as_np=False):
        x_hidden = self.encoder(categorical_data, continuous_data)
        
        x_hidden = self.flatten(x_hidden)
        
        mu, logvar = self.hidden2mu(x_hidden), self.hidden2logvar(x_hidden)
        print("reparam")
        z = self.reparam(mu, logvar)
        
        if as_np: return to_np(z)
        else: return z
        
    def get_posteriors(self, categorical_data, continuous_data):

        return self.encode(continuous_data, categorical_data)

    def get_z(self, categorical_data, continuous_data):
        """Encode a batch of data points, x, into their z representations."""

        mu, logvar = self.encode(categorical_data, continuous_data)
        return self.reparam(mu, logvar)

    def reparam(self, mu, logvar):
        """Reparameterisation trick to sample z values.
        This is stochastic during training, and returns the mode during evaluation."""

        if self.training:
            print("sample")
            # convert logarithmic variance to standard deviation representation
            std = logvar.mul(0.5).exp_()
            # create normal distribution as large as the data
            eps = Variable(std.data.new(std.size()).normal_())
            # scale by learned mean and standard deviation
            return eps.mul(std).add_(mu)
        else:
            return mu
        
#     def decode(self, categorical_data, continuous_data, unflatten_dims=None, as_np=False):
        
#         if not unflatten_dims: 
                
#         continuous_data = self.unflatten(continuous_data, unflatten_dims)
        
#         x = self.decoder(categorical_data, continuous_data)
        
#         if as_np: return to_np(x)
        
#         else: return x
        

In [ ]:
x = torch.randn((3,10), requires_grad=True)
x.shape

torch.Size([3, 10])

In [ ]:
enc = MultiLayerPerceptron(ann_structure)
dec = MultiLayerPerceptron(ann_structure[::-1])

vae = VariationalAutoencoder(enc, dec, ann_structure[-1], ann_structure[-1])

In [ ]:
1/0 # why is this not sampling???

ZeroDivisionError: division by zero

In [ ]:
vae.training = True
vae.forward(None, x)

reparam
sample


tensor([[-0.4814, -0.3851,  1.4582,  0.5811,  1.9413, -0.9969,  0.0164,  0.4830,
         -0.5062, -0.3639],
        [-0.0331, -0.1893,  0.7329, -1.1195,  2.5510,  0.7493,  0.6637,  0.0988,
         -0.2158,  0.4984],
        [ 0.8742, -0.3657, -1.0747,  0.2848,  0.2856,  0.2524, -0.3153,  0.1281,
         -1.3582,  0.3956]], requires_grad=True)